#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo


In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [6]:
df_ks = df_ks.rename({"usd pledged":"usd_pledged"}, axis='columns')

In [7]:
df_ks = df_ks.drop(columns=['ID', 'currency', 'deadline', 'backers'])

In [8]:
df_ks['goal'] = pd.to_numeric(df_ks['goal'],errors = 'coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors = 'coerce')
df_ks['usd_pledged'] = pd.to_numeric(df_ks['usd_pledged'],errors = 'coerce')

In [9]:
df_ks.dtypes

name                 object
category             object
main_category        object
goal                float64
launched             object
pledged             float64
state                object
country              object
usd_pledged         float64
usd_pledged_real    float64
dtype: object

### Importer les données

In [10]:
collection.delete_many({})

In [11]:
dict = df_ks.to_dict('records')

In [12]:
collection.insert_many(dict)

## Question 1  

In [13]:
#prendre les 5 pledged les plus grand
cursor = collection.find().sort([("pledged",-1)]).limit(5)
list(cursor)


[{'_id': ObjectId('61f6eab462275959867e6ae0'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 50000.0,
  'launched': '2014-07-08 10:14:37',
  'pledged': 13285226.36,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('61f6eab462275959867ec5d9'),
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 1000000.0,
  'launched': '2016-05-24 15:49:52',
  'pledged': 12779843.49,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('61f6eab462275959867e7717'),
  'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 800000.0,
  'launched': '2017-10-10 12:52:25',
  'pledged': 10035296.0,
  'state': 'succes

## Question 2

In [14]:
collection.count_documents({"state" : "successful"})


53040

## Question 3

In [15]:
cursor = collection.aggregate([{"$group" : {"_id" : "$category", "NombreParCatégorie" : {"$sum" : 1}}}])
list(cursor)

[{'_id': 'Camera Equipment', 'NombreParCatégorie': 165},
 {'_id': 'Wearables', 'NombreParCatégorie': 508},
 {'_id': 'Childrenswear', 'NombreParCatégorie': 192},
 {'_id': 'Art Books', 'NombreParCatégorie': 1065},
 {'_id': 'Product Design', 'NombreParCatégorie': 8886},
 {'_id': 'Faith', 'NombreParCatégorie': 439},
 {'_id': 'Events', 'NombreParCatégorie': 322},
 {'_id': 'Places', 'NombreParCatégorie': 283},
 {'_id': 'Literary Journals', 'NombreParCatégorie': 118},
 {'_id': 'Fashion', 'NombreParCatégorie': 3379},
 {'_id': 'Accessories', 'NombreParCatégorie': 1204},
 {'_id': 'Apparel', 'NombreParCatégorie': 2827},
 {'_id': 'Apps', 'NombreParCatégorie': 2535},
 {'_id': 'Experimental', 'NombreParCatégorie': 357},
 {'_id': 'Webcomics', 'NombreParCatégorie': 259},
 {'_id': 'Horror', 'NombreParCatégorie': 525},
 {'_id': 'R&B', 'NombreParCatégorie': 172},
 {'_id': 'Hardware', 'NombreParCatégorie': 1431},
 {'_id': 'Classical Music', 'NombreParCatégorie': 1064},
 {'_id': 'Printing', 'NombreParCatég

## Question 4

In [16]:
collection.count_documents({"$and":[{"launched":{"$lt": "2016"}}, {"country":"FR"}]})

330

## Question 5

In [17]:
cursor = collection.find({"$and":[{"usd_pledged_real":{"$gte": 200000}}, {"country":"US"}]})
list(cursor)

[{'_id': ObjectId('61f6eab362275959867d3fea'),
  'name': 'The uKeg Pressurized Growler for Fresh Beer',
  'category': 'Drinks',
  'main_category': 'Food',
  'goal': 75000.0,
  'launched': '2014-10-15 06:34:48',
  'pledged': 1559525.68,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 1559525.68,
  'usd_pledged_real': 1559525.68},
 {'_id': ObjectId('61f6eab362275959867d407a'),
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 30000.0,
  'launched': '2015-07-21 19:01:41',
  'pledged': 692912.0,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('61f6eab362275959867d4128'),
  'name': 'Legion Solar - A Better Way to Energy Independence',
  'category': 'Technology',
  'main_category': 'Technology',
  'goal': 50000.0,
  'launched': '2016-05-03 16:00:22',
  'pledged': 277524.0,
  'state': 'successful',
  'country': 'US',
  'u

## Question 6 

In [18]:
collection.create_index([("name",  "text")])

'name_text'

In [19]:
collection.count_documents({ "$text": { "$search": "Sport" } })

318